Objective
=========

To train a neural network to segment neurons, we first need data to train it on.
Our approach to this is to simulate imagery from a microscope using the geometry of digitized neuron models.
To do this, we source the models from [NeuroMorpho](https://neuromorpho.org) and load them into the microscope simulator.
This repository comes with a set of SWC files (the neuron models) from NeuroMorpho and can be used for this purpose.

In [1]:
from pathlib import Path
from PIL import Image
import neuroscope

def get_swc_list() -> list[Path]:
    entries = []
    for entry in Path('../data/swc').glob('*/*.swc'):
        entries.append(entry)
    return entries

def open_models(swc_list: list[Path]) -> list[neuroscope.SWCModel]:
    models: list[neuroscope.SWCModel] = []
    for swc_path in swc_list:
        model = neuroscope.SWCModel()
        if not model.load_from_file(str(swc_path)):
            print(f'failed to open {swc_path}, skipping.')
            pass
        models.append(model)
    return models

def render_models(swc_models: list[neuroscope.SWCModel], res: tuple[int, int] = (512, 512), fov=600):
    out_dir = Path('../data/imagery')
    out_dir.mkdir(exist_ok=True)
    debug_scope = neuroscope.DebugMicroscope(res[0], res[1], vertical_fov=fov)
    scope = neuroscope.GenericFluorescentMicroscope(res[0], res[1], vertical_fov=fov)
    counter = 0
    for model in swc_models:

        # segmentation
        debug_scope.capture(model)
        seg_buffer = debug_scope.copy_rgb_buffer()
        seg_path = out_dir / f'seg_{counter:05}.png'
        Image.frombytes('RGB', res, seg_buffer).save(seg_path)

        # real
        scope.capture(model)
        real_buffer = scope.copy_buffer()
        real_path = out_dir / f'{counter:05}.png'
        Image.frombytes('L', res, real_buffer).save(real_path)

        counter += 1
    pass
swc_paths = get_swc_list()
swc_models = open_models(swc_paths)
render_models(swc_models)
